In [1]:
import numpy as np
import pandas as pd
import re
from currency_converter import CurrencyConverter

In [2]:
df = pd.read_csv("raw_data.csv")

/home/daniel/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
df.iloc[df.averageRating < 6] = np.nan

In [22]:
short = df[-100:]
zeros = np.zeros(len(short))
short.loc[:,"endYear"] = zeros

/home/daniel/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [23]:
from IPython.display import display, HTML
short
#mask = (currencies == "₹")
#display(HTML(df[mask].to_html()))

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Budget,Gross US & Canada,Opening weekend US & Canada,Gross worldwide,Rating,User reviews,Critic reviews,directors,writers
110055,NaN,movie,Charter,Charter,0.0,2020,0.0,94,Drama,0.0,1117.0,NaN,NaN,NaN,NaN,NaN,7,18.0,nm1840544,nm1840544
110056,NaN,movie,Children of the Sea,Kaijû no kodomo,0.0,2019,0.0,111,"Adventure,Animation,Drama",0.0,2482.0,NaN,NaN,NaN,"$5,092,603",Not Rated,23,46.0,nm2268014,"nm6226680,nm11218783"
110057,NaN,movie,Paradise War: The Story of Bruno Manser,Bruno Manser - Die Stimme des Regenwaldes,0.0,2019,0.0,142,"Adventure,Biography,Drama",0.0,368.0,NaN,NaN,NaN,NaN,NaN,5.0,10.0,nm1305198,"nm9503680,nm1305198,nm0865453"
110058,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110059,NaN,movie,#Anne Frank Parallel Stories,#AnneFrank - Parallel Stories,0.0,2019,0.0,92,"Drama,History",0.0,1411.0,NaN,NaN,NaN,"$327,285",NaN,34,10.0,"nm10509355,nm9921790",\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110150,NaN,movie,Sokagin Çocuklari,Sokagin Çocuklari,0.0,2019,0.0,98,"Drama,Family",0.0,218.0,NaN,NaN,NaN,"$2,833",NaN,NaN,NaN,nm1832009,"nm1832009,nm1902682"
110151,NaN,movie,Life Without Sara Amat,La vida sense la Sara Amat,0.0,2019,0.0,74,Drama,0.0,137.0,NaN,NaN,NaN,"$59,794",NaN,NaN,NaN,nm1716653,"nm3678448,nm9361716"
110152,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110153,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
budget_raw = df["Budget"].to_numpy()
currencies = np.zeros(len(budget_raw), dtype='U10')
value = np.zeros(len(budget_raw), dtype=int)
year = df["startYear"].to_numpy()

for i in range(len(budget_raw)):
    string = budget_raw[i]
    
    if not pd.isna(string):

        value[i] = int("".join(re.findall(r'[\d]+', string)))
        currencies[i] = re.findall(r'[^{\d,\xa0}]+', string)[0]

In [77]:
# USD
df_usd = pd.read_csv("./inflation_data/usd.csv")
np_amount = df_usd["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "$":
        target_year = int(year[i]) if int(year[i]) >= 1910 else 1910
        inflation_factor = current_amount/df_usd[df_usd.year == target_year].amount
        currencies[i] = "usd"
        value[i] = value[i] * inflation_factor
del df_usd

In [78]:
# EUR

c = CurrencyConverter()

df_eur = pd.read_csv("./inflation_data/eur.csv")
np_amount = df_eur["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "€":
        target_year = int(year[i]) if int(year[i]) >= 1996 else 1996
        inflation_factor = current_amount/df_eur[df_eur.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'EUR', 'USD')
        currencies[i] = "usd"
        
        
del df_eur

In [79]:
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 22257, '₹': 1198, '£': 955, 'CA$': 787, 'A$': 320, 'SEK': 186, 'NOK': 162, 'R$': 149, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'DKK': 122, 'RUR': 122, 'EGP': 96, 'IRR': 84, 'CN¥': 81, 'TRL': 81, 'HUF': 81, 'PLN': 76, '¥': 59, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [80]:
# INR

c = CurrencyConverter()

df_inr = pd.read_csv("./inflation_data/inr.csv")
np_amount = df_inr["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "₹":
        target_year = int(year[i]) if int(year[i]) >= 1960 else 1960
        inflation_factor = current_amount/df_inr[df_inr.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'INR', 'USD')
        currencies[i] = "usd"
        
        
del df_inr

In [81]:
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 23455, '£': 955, 'CA$': 787, 'A$': 320, 'SEK': 186, 'NOK': 162, 'R$': 149, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'DKK': 122, 'RUR': 122, 'EGP': 96, 'IRR': 84, 'CN¥': 81, 'TRL': 81, 'HUF': 81, 'PLN': 76, '¥': 59, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [82]:
# gbp

c = CurrencyConverter()

df_gbp = pd.read_csv("./inflation_data/gbp.csv")
np_amount = df_gbp["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "£":
        target_year = int(year[i]) if int(year[i]) >= 1910 else 1910
        inflation_factor = current_amount/df_gbp[df_gbp.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'GBP', 'USD')
        currencies[i] = "usd"

        
del df_gbp

In [83]:
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 24410, 'CA$': 787, 'A$': 320, 'SEK': 186, 'NOK': 162, 'R$': 149, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'DKK': 122, 'RUR': 122, 'EGP': 96, 'IRR': 84, 'CN¥': 81, 'TRL': 81, 'HUF': 81, 'PLN': 76, '¥': 59, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [84]:
# cad

c = CurrencyConverter()

df_cad = pd.read_csv("./inflation_data/cad.csv")
np_amount = df_cad["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "CA$":
        target_year = int(year[i]) if int(year[i]) >= 1915 else 1915
        inflation_factor = current_amount/df_cad[df_cad.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'CAD', 'USD')
        currencies[i] = "usd"

        
del df_cad

In [85]:
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 25197, 'A$': 320, 'SEK': 186, 'NOK': 162, 'R$': 149, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'DKK': 122, 'RUR': 122, 'EGP': 96, 'IRR': 84, 'CN¥': 81, 'TRL': 81, 'HUF': 81, 'PLN': 76, '¥': 59, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [86]:
# aud

c = CurrencyConverter()

df_aud = pd.read_csv("./inflation_data/aud.csv")
np_amount = df_aud["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "A$":
        target_year = int(year[i]) if int(year[i]) >= 1915 else 1915
        inflation_factor = current_amount/df_aud[df_aud.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'AUD', 'USD')
        currencies[i] = "usd"

        
del df_aud

In [87]:
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 25517, 'SEK': 186, 'NOK': 162, 'R$': 149, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'DKK': 122, 'RUR': 122, 'EGP': 96, 'IRR': 84, 'CN¥': 81, 'TRL': 81, 'HUF': 81, 'PLN': 76, '¥': 59, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [88]:
# sek

c = CurrencyConverter()

df_sek = pd.read_csv("./inflation_data/sek.csv")
np_amount = df_sek["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "SEK":
        target_year = int(year[i]) if int(year[i]) >= 1960 else 1960
        inflation_factor = current_amount/df_sek[df_sek.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'SEK', 'USD')
        currencies[i] = "usd"

        
del df_sek

In [89]:
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 25703, 'NOK': 162, 'R$': 149, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'DKK': 122, 'RUR': 122, 'EGP': 96, 'IRR': 84, 'CN¥': 81, 'TRL': 81, 'HUF': 81, 'PLN': 76, '¥': 59, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [90]:
# nok

c = CurrencyConverter()

df_nok = pd.read_csv("./inflation_data/nok.csv")
np_amount = df_nok["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "NOK":
        target_year = int(year[i]) if int(year[i]) >= 1960 else 1960
        inflation_factor = current_amount/df_nok[df_nok.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'NOK', 'USD')
        currencies[i] = "usd"

        
del df_nok

In [91]:
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 25865, 'R$': 149, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'DKK': 122, 'RUR': 122, 'EGP': 96, 'IRR': 84, 'CN¥': 81, 'TRL': 81, 'HUF': 81, 'PLN': 76, '¥': 59, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [92]:
# zar

c = CurrencyConverter()

df_zar = pd.read_csv("./inflation_data/zar.csv")
np_amount = df_zar["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "R$":
        target_year = int(year[i]) if int(year[i]) >= 1960 else 1960
        inflation_factor = current_amount/df_zar[df_zar.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'ZAR', 'USD')
        currencies[i] = "usd"

        
del df_zar
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 26014, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'DKK': 122, 'RUR': 122, 'EGP': 96, 'IRR': 84, 'CN¥': 81, 'TRL': 81, 'HUF': 81, 'PLN': 76, '¥': 59, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [93]:
# dkk

c = CurrencyConverter()

df_dkk = pd.read_csv("./inflation_data/dkk.csv")
np_amount = df_dkk["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "DKK":
        target_year = int(year[i]) if int(year[i]) >= 1960 else 1960
        inflation_factor = current_amount/df_dkk[df_dkk.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'DKK', 'USD')
        currencies[i] = "usd"

        
del df_dkk
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 26136, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'RUR': 122, 'EGP': 96, 'IRR': 84, 'CN¥': 81, 'TRL': 81, 'HUF': 81, 'PLN': 76, '¥': 59, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [98]:
# rub

c = CurrencyConverter()

df_rub = pd.read_csv("./inflation_data/rub.csv")
np_amount = df_rub["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "RUR":
        target_year = int(year[i]) if int(year[i]) >= 1993 else 1993
        inflation_factor = current_amount/df_rub[df_rub.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'RUB', 'USD')
        currencies[i] = "usd"

        
del df_rub
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 26258, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'EGP': 96, 'IRR': 84, 'CN¥': 81, 'TRL': 81, 'HUF': 81, 'PLN': 76, '¥': 59, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [100]:
# cny

c = CurrencyConverter()

df_cny = pd.read_csv("./inflation_data/cny.csv")
np_amount = df_cny["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "CN¥":
        target_year = int(year[i]) if int(year[i]) >= 1987 else 1987
        inflation_factor = current_amount/df_cny[df_cny.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'CNY', 'USD')
        currencies[i] = "usd"

        
del df_cny
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 26339, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'EGP': 96, 'IRR': 84, 'TRL': 81, 'HUF': 81, 'PLN': 76, '¥': 59, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [103]:
# try

c = CurrencyConverter()

df_try = pd.read_csv("./inflation_data/try.csv")
np_amount = df_try["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "TRL":
        target_year = int(year[i]) if int(year[i]) >= 1960 else 1960
        inflation_factor = current_amount/df_try[df_try.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'TRY', 'USD')
        currencies[i] = "usd"

        
del df_try
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 26420, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'EGP': 96, 'IRR': 84, 'HUF': 81, 'PLN': 76, '¥': 59, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [105]:
# huf

c = CurrencyConverter()

df_huf = pd.read_csv("./inflation_data/huf.csv")
np_amount = df_huf["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "HUF":
        target_year = int(year[i]) if int(year[i]) >= 1973 else 1973
        inflation_factor = current_amount/df_huf[df_huf.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'HUF', 'USD')
        currencies[i] = "usd"

        
del df_huf
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 26501, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'EGP': 96, 'IRR': 84, 'PLN': 76, '¥': 59, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [107]:
# pln

c = CurrencyConverter()

df_pln = pd.read_csv("./inflation_data/jpy.csv")
np_amount = df_pln["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "PLN":
        target_year = int(year[i]) if int(year[i]) >= 1971 else 1971
        inflation_factor = current_amount/df_pln[df_pln.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'PLN', 'USD')
        currencies[i] = "usd"

        
del df_pln
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 26577, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'EGP': 96, 'IRR': 84, '¥': 59, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [109]:
# jpy

c = CurrencyConverter()

df_jpy = pd.read_csv("./inflation_data/jpy.csv")
np_amount = df_jpy["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "¥":
        target_year = int(year[i]) if int(year[i]) >= 1960 else 1960
        inflation_factor = current_amount/df_jpy[df_jpy.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'JPY', 'USD')
        currencies[i] = "usd"

        
del df_jpy
print(Counter(list(currencies)))

Counter({'': 82075, 'usd': 26636, 'FRF': 144, 'DEM': 139, 'FIM': 122, 'EGP': 96, 'IRR': 84, 'HK$': 51, 'CZK': 45, '₩': 44, 'ITL': 42, 'MX$': 42, 'ESP': 41, 'NZ$': 40, 'BDT': 37, 'MYR': 37, 'NLG': 36, 'CHF': 33, 'SGD': 33, 'IDR': 33, 'THB': 29, 'ARS': 24, 'ZAR': 22, 'ROL': 21, 'UAH': 20, 'ISK': 19, 'NT$': 19, 'PKR': 18, 'PTE': 15, 'DOP': 14, '₱': 13, '₪': 11, 'HRK': 8, 'AZM': 8, 'BEF': 7, 'NPR': 7, 'BGL': 7, 'LKR': 7, 'EEK': 6, 'RON': 6, 'COP': 5, '₫': 5, 'CLP': 4, 'VEB': 4, 'LVL': 4, 'NGN': 4, 'GEL': 4, 'SIT': 3, 'YUM': 3, 'LTL': 3, 'MNT': 3, 'ALL': 2, 'SAR': 2, 'AMD': 2, 'AED': 2, 'XAU': 1, 'PYG': 1, 'IEP': 1, 'JMD': 1, 'ATS': 1, 'BND': 1, 'GRD': 1, 'SKK': 1, 'MTL': 1, 'IQD': 1, 'TTD': 1, 'JOD': 1, 'ZWD': 1, 'MVR': 1})


In [108]:
pd.read_csv("./inflation_data/jpy.csv")

,year,amount,inflation rate
0,1960,1.00,0.04
1,1961,1.05,0.05
2,1962,1.13,0.07
3,1963,1.20,0.07
4,1964,1.25,0.04
...,...,...,...
58,2018,5.53,0.01
59,2019,5.55,0.00
60,2020,5.55,-0.00
61,2021,5.55,-0.00
